In [1]:
from utils import AMAEConfig
from model import AudioMAE
import librosa
import os
import torch
from torchinfo import summary


In [2]:
test_cfg = AMAEConfig(cfg_path=r'E:\project\python\Audio_MAE_Base_ST\config\auto_encoder.yaml')
test_m = AudioMAE(test_cfg)
test_f = os.path.join('datas', 'test', 'test_10s.wav')
signal, _ = librosa.load(test_f, sr=32000, mono=True)
model_input = torch.Tensor(signal).unsqueeze(0)

In [3]:
summary(test_m, input_size=(4, 320000), depth=5, col_names=("input_size", "output_size", "num_params", "trainable", "kernel_size"))

Layer (type:depth-idx)                                  Input Shape               Output Shape              Param #                   Trainable                 Kernel Shape
AudioMAE                                                [4, 320000]               [4, 1, 1024, 64]          --                        Partial                   --
├─STEncoder: 1-1                                        [4, 320000]               --                        --                        Partial                   --
│    └─Spectrogram: 2-1                                 [4, 320000]               [4, 1, 1001, 513]         --                        False                     --
│    │    └─STFT: 3-1                                   --                        --                        --                        False                     --
│    │    │    └─Conv1d: 4-1                            [4, 1, 321024]            [4, 513, 1001]            (525,312)                 False                     [1024]
│    │  

In [ ]:
output = test_m(model_input)
output.shape

In [ ]:
output[:, 0::2, :, :].shape

In [ ]:
a = torch.tensor(list(range(24)), dtype=torch.int32)
b = a + 10
a, b = a.reshape(2, 2, 2, 3), b.reshape(2, 2, 2, 3)
a, b


In [ ]:
c = x = torch.cat([a, b], -1)
c

In [ ]:
a = [10, 100, 1000, 10000]
a = torch.tensor(a, dtype=torch.int32)
a = torch.stack([a, a+1], dim = 1)

b = a * 2
c = a * 3
d = a * 4
a.shape, a, b, c, d

In [ ]:
x1 = torch.stack([a, c], dim = 1).reshape(2, 4, 2)
x1.shape, x1

In [ ]:
x2 = torch.stack([b, d], dim = 1).reshape(2, 4, 2)
x2.shape, x2

In [ ]:
x = torch.stack([x1, x2], dim = 1).reshape(4, 4, 2)
x.shape, x

In [ ]:
import torch
H1, W1 = 8, 8
ori = torch.randint(high=100, size=(12, 64, 15), dtype=torch.int32)
B1, L1, C1 = ori.shape
B1, L1, C1, H1, W1

In [ ]:
m = ori.view(B1, H1, W1, C1)
m0 = m[:, 0::2, 0::2, :]  # [B, H/2, W/2, C]
m1 = m[:, 1::2, 0::2, :]  # [B, H/2, W/2, C]
m2 = m[:, 0::2, 1::2, :]  # [B, H/2, W/2, C]
m3 = m[:, 1::2, 1::2, :]  # [B, H/2, W/2, C]
o = torch.cat([m0, m1, m2, m3], -1)  # [B, H/2, W/2, 4*C]
o = o.view(B1, -1, 4 * C1)  # [B, H/2*W/2, 4*C]
o.shape

In [ ]:
H2, W2 = 4, 4
B2, L2, C2 = o.shape

n0 = o[..., :C2//4]
n1 = o[..., C2//4:C2//4 * 2]
n2 = o[..., C2//4 * 2:C2//4 * 3]
n3 = o[..., C2//4 * 3:C2//4 * 4]

a = torch.stack([n0, n2], dim=2).reshape(B2, H2, 2 * W2, C2//4)
b = torch.stack([n1, n3], dim=2).reshape(B2, H2, 2 * W2, C2//4)
final = torch.stack([a, b], dim=2).reshape(B2, 4*L2, C2//4)
final.shape

In [ ]:
(final == ori).sum() == B1 * L1 * C1

In [6]:
def reshape_wav2img(x, freq_ratio):
    x = x.permute(0, 1, 3, 2).contiguous()
    x = x.reshape(x.shape[0], x.shape[1], x.shape[2], freq_ratio, x.shape[3] // freq_ratio)
    x = x.permute(0, 1, 3, 2, 4).contiguous()
    x = x.reshape(x.shape[0], x.shape[1], x.shape[2] * x.shape[3], x.shape[4])
    return x  # [B, 1, f, T]


def reshape_img2wav(x, freq_ratio):
    batch, channel, freq, time = x.shape
    ori_freq_bin = freq // freq_ratio
    x = x.reshape(batch, channel, freq // ori_freq_bin, ori_freq_bin, time)
    x = x.permute(0, 1, 3, 2, 4).contiguous().reshape(batch, channel, ori_freq_bin, -1)
    x = x.permute(0, 1, 3, 2).contiguous()
    return x

In [9]:
a = torch.randint(high=1000000, size=(4, 1, 1024, 64), dtype=torch.int32)
a.shape

torch.Size([4, 1, 1024, 64])

In [11]:
b = reshape_wav2img(a, 4)
b.shape

torch.Size([4, 1, 256, 256])

In [13]:
c = reshape_wav2img(b, 4)
c.shape

torch.Size([4, 1, 1024, 64])

In [17]:
(a == c).sum() == 4 * 1 * 1024 * 64

tensor(True)

In [3]:
import torch
x = torch.randint(high=100, size=(4, 1, 3, 2), dtype=torch.int32)
x

tensor([[[[41, 77],
          [43, 73],
          [ 8, 31]]],


        [[[46, 78],
          [94, 36],
          [53, 26]]],


        [[[91, 54],
          [11,  9],
          [40, 90]]],


        [[[10, 92],
          [46, 20],
          [70,  1]]]], dtype=torch.int32)

In [4]:
B, C, T, F = x.shape
L = T * F
x = x.reshape(B, C, -1)
x

tensor([[[41, 77, 43, 73,  8, 31]],

        [[46, 78, 94, 36, 53, 26]],

        [[91, 54, 11,  9, 40, 90]],

        [[10, 92, 46, 20, 70,  1]]], dtype=torch.int32)

In [7]:
len_keep = 2
noise = torch.rand(B, C, L, device=x.device)  # noise in [0, 1]

# sort noise for each sample
ids_shuffle = torch.argsort(noise, dim=2)  # ascend: small is keep, large is remove
ids_restore = torch.argsort(ids_shuffle, dim=2)
mask = torch.zeros([B, C, L], device=x.device)
mask[:, :, :len_keep] = 1
mask = torch.gather(mask, dim=2, index=ids_restore)
mask

tensor([[[0., 1., 1., 0., 0., 0.]],

        [[1., 1., 0., 0., 0., 0.]],

        [[0., 0., 0., 1., 0., 1.]],

        [[1., 0., 0., 1., 0., 0.]]])

In [58]:
mask = mask.reshape(B, C, T, F)
x = x.reshape(B, C, T, F)
mask, x

(tensor([[[[1., 0.],
           [0., 1.],
           [0., 0.]]],
 
 
         [[[1., 0.],
           [1., 0.],
           [0., 0.]]],
 
 
         [[[0., 0.],
           [1., 0.],
           [1., 0.]]],
 
 
         [[[0., 0.],
           [0., 1.],
           [1., 0.]]]]),
 tensor([[[[45, 94],
           [85,  0],
           [63, 53]]],
 
 
         [[[72, 38],
           [34, 12],
           [68,  7]]],
 
 
         [[[49,  2],
           [ 2,  2],
           [23,  3]]],
 
 
         [[[17, 62],
           [44, 78],
           [46, 94]]]], dtype=torch.int32))

In [59]:
x = torch.mul(x, mask)
x

tensor([[[[45.,  0.],
          [ 0.,  0.],
          [ 0.,  0.]]],


        [[[72.,  0.],
          [34.,  0.],
          [ 0.,  0.]]],


        [[[ 0.,  0.],
          [ 2.,  0.],
          [23.,  0.]]],


        [[[ 0.,  0.],
          [ 0., 78.],
          [46.,  0.]]]])

In [32]:
noise = torch.rand(N, L, device=x.device)
ids_shuffle = torch.argsort(noise, dim=1)
ids_restore = torch.argsort(ids_shuffle, dim=1)
ids_shuffle

tensor([[1, 3, 0, 2],
        [0, 1, 3, 2]])

In [33]:
mask = torch.zeros([N, L], device=x.device)
mask[:, :len_keep] = 1
mask = torch.gather(mask, dim=1, index=ids_restore)
mask = mask.unsqueeze(-1).repeat(1, 1, 3)
mask.shape, mask

(torch.Size([2, 4, 3]),
 tensor([[[0., 0., 0.],
          [1., 1., 1.],
          [0., 0., 0.],
          [0., 0., 0.]],
 
         [[1., 1., 1.],
          [0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]]]))

In [34]:
torch.mul(x, mask)

tensor([[[0.0000, 0.0000, 0.0000],
         [0.8291, 0.2354, 0.0982],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000]],

        [[0.1519, 0.8590, 0.0119],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000]]])

In [30]:
noise = torch.rand(N, L, device=x.device)  # noise in [0, 1]
        
# sort noise for each sample
ids_shuffle = torch.argsort(noise, dim=1)  # ascend: small is keep, large is remove
ids_restore = torch.argsort(ids_shuffle, dim=1)

# keep the first subset
ids_keep = ids_shuffle[:, :len_keep]
x_masked = torch.gather(x, dim=1, index=ids_keep.unsqueeze(-1).repeat(1, 1, D))

# generate the binary mask: 0 is keep, 1 is remove
mask = torch.ones([N, L], device=x.device)
mask[:, :len_keep] = 0
# unshuffle to get the binary mask
mask = torch.gather(mask, dim=1, index=ids_restore)

In [31]:
mask

tensor([[1., 1., 1., 1., 1., 1., 1., 0., 1., 0.]])

In [32]:
x_masked

tensor([[[390, 631, 208, 625, 228],
         [482, 543, 141, 933, 960]]], dtype=torch.int32)

In [12]:
import torchaudio
import torch
import os
import librosa
from torchlibrosa.stft import Spectrogram, LogmelFilterBank

test_f = os.path.join('datas', 'test', 'test_10s.wav')
raw_signal, _ = librosa.load(test_f, sr=32000, mono=True)
# fband_new = torchaudio.compliance.kaldi.fbank()

In [13]:
def fband_ori(wav):
    spectrogram_extractor = Spectrogram(n_fft=1024, hop_length=320, win_length=1024)
    # log_mel feature extractor
    logmel_extractor = LogmelFilterBank(sr=32000, n_fft=1024, n_mels=64, fmin=50.)
    x = spectrogram_extractor(wav)
    x = logmel_extractor(x)
    return x

In [15]:
fband1 = fband_ori(torch.Tensor(raw_signal).unsqueeze(0))
fband1.shape, fband1

(torch.Size([1, 1, 1001, 64]),
 tensor([[[[-72.1170, -72.1170, -72.1170,  ..., -72.1170, -72.1170, -72.1170],
           [-72.1170, -72.1170, -72.1170,  ..., -72.1170, -72.1170, -72.1170],
           [-72.1170, -72.1170, -72.1170,  ..., -72.1170, -72.1170, -72.1170],
           ...,
           [-32.4773, -36.4364, -46.5780,  ..., -72.1170, -72.1170, -72.1170],
           [-33.5813, -34.5268, -41.4278,  ..., -72.1170, -72.1170, -72.1170],
           [-34.2101, -28.8399, -31.1258,  ..., -72.1170, -72.1170, -72.1170]]]]))

In [21]:
fband2 = torchaudio.compliance.kaldi.fbank(torch.Tensor(raw_signal).unsqueeze(0), frame_length=32, frame_shift=10, low_freq=20., sample_frequency=32000, window_type='hanning', dither=0.0, htk_compat=True)
fband2.shape, fband2

(torch.Size([997, 23]),
 tensor([[-15.9424, -15.9424, -15.9424,  ..., -15.9424, -15.9424, -15.9424],
         [-15.9424, -15.9424, -15.9424,  ..., -15.9424, -15.9424, -15.9424],
         [-15.9424, -15.9424, -15.9424,  ..., -15.9424, -15.9424, -15.9424],
         ...,
         [-10.9924, -11.2170, -11.5014,  ..., -10.0303, -11.8588, -11.4869],
         [-10.5070, -11.6606, -12.5122,  ..., -11.3630, -11.9011, -11.3635],
         [ -9.7052, -11.4238, -12.2474,  ..., -11.3084, -11.5466, -11.3210]]))

In [ ]:
# audiomae:fbank = torchaudio.compliance.kaldi.fbank(waveform, htk_compat=True, sample_frequency=sr, use_energy=False,
#                                                   window_type='hanning', num_mel_bins=self.melbins, dither=0.0, frame_shift=10)

In [ ]:
# EAT:feats = torchaudio.compliance.kaldi.fbank(feats, htk_compat=True, sample_frequency=curr_sample_rate, use_energy=False,
#                                                   window_type='hanning', num_mel_bins=128, dither=0.0, frame_shift=10).unsqueeze(dim=0)